# 01 - Coleta e Pré-Processamento

Este notebook carrega os dados brutos e realiza o pré-processamento completo, incluindo preenchimento de lacunas e geração de features ricas.

In [6]:
# Importar bibliotecas
import pandas as pd
import numpy as np

# Função de pré-processamento
def gerar_demanda_completa(path_raw_pedidos: str, path_raw_eventos: str, path_output: str) -> pd.DataFrame:
    # Carregar pedidos
    pedidos = pd.read_csv(path_raw_pedidos, parse_dates=["date"])
    pedidos = pedidos[pedidos["order_status"] == "Delivered"].copy()
    pedidos["date"] = pd.to_datetime(pedidos["date"].dt.date)

    # Agrupar demanda nacional por dia
    demanda_nacional = (
        pedidos
        .groupby("date", as_index=False)
        .agg(total_nacional=("quantity", "sum"))
        .set_index("date")
    )

    # Preencher datas faltantes
    datas_completas = pd.date_range(start=demanda_nacional.index.min(),
                                    end=demanda_nacional.index.max(),
                                    freq="D")
    demanda_nacional = demanda_nacional.reindex(datas_completas, fill_value=0)
    demanda_nacional.index.name = "date"

    # Carregar eventos
    eventos = pd.read_csv(path_raw_eventos, parse_dates=["date"])
    eventos["date"] = pd.to_datetime(eventos["date"].dt.date)
    datas_com_evento = (
        eventos[["date"]]
        .drop_duplicates()
        .assign(evento_na_data=1)
    )

    # Juntar eventos como exógena
    demanda_nacional = demanda_nacional.merge(
        datas_com_evento, left_index=True, right_on="date", how="left"
    ).set_index("date").fillna({"evento_na_data": 0})

    # Renomear coluna para 'y'
    demanda_nacional = demanda_nacional.rename(columns={"total_nacional": "y"})

    # Função para criar features ricas
    def criar_features_ricos(df: pd.DataFrame):
        df = df.copy()

        # Criação de lags
        for lag in [1, 7, 14, 21, 30]:
            df[f"lag_{lag}"] = df["y"].shift(lag)

        # Médias móveis
        df["rolling_mean_7"] = df["y"].rolling(window=7).mean()
        df["rolling_mean_30"] = df["y"].rolling(window=30).mean()

        # Novas features derivadas
        df["diff_rolling_mean_7_30"] = df["rolling_mean_7"] - df["rolling_mean_30"]
        df["rolling_std_7"] = df["y"].rolling(window=7).std()
        df["zscore_demanda"] = (df["y"] - df["rolling_mean_30"]) / df["rolling_mean_30"].std()
        df["week_of_month"] = df.index.day.map(lambda d: (d - 1) // 7 + 1)

        # Variáveis de calendário
        df["day_of_week"] = df.index.dayofweek
        df["is_weekend"] = (df["day_of_week"] >= 5).astype(int)
        df["month"] = df.index.month
        df["day"] = df.index.day
        df["is_month_start"] = df.index.is_month_start.astype(int)
        df["is_month_end"] = df.index.is_month_end.astype(int)

        # Dias até próximo feriado (requer 'holidays')
        try:
            import holidays
            br_holidays = holidays.Brazil()
            df["days_until_holiday"] = [
                min([(h - d).days for h in br_holidays if h > d], default=999)
                for d in df.index
            ]
        except ImportError:
            print("⚠️ Biblioteca 'holidays' não instalada. 'days_until_holiday' não será calculada.")
            df["days_until_holiday"] = 999  # valor padrão

        return df.dropna()

    # Gerar DataFrame com features
    demanda_features = criar_features_ricos(demanda_nacional)

    # Salvar em CSV
    demanda_features.to_csv(path_output, index=True)
    return demanda_features

# Executar pré-processamento
df_demanda_features = gerar_demanda_completa(
    path_raw_pedidos="../data/raw/dados_pedidos.csv",
    path_raw_eventos="../data/raw/dados_contextuais.csv",
    path_output="../data/processed/demanda_features.csv"
)

# Mostrar primeiras linhas
print(df_demanda_features.head())


              y  evento_na_data  lag_1  lag_7  lag_14  lag_21  lag_30  \
date                                                                    
2023-01-31   66             0.0  161.0   70.0   212.0   195.0   259.0   
2023-02-01  235             1.0   66.0  139.0    36.0    85.0   238.0   
2023-02-02  257             0.0  235.0  269.0   136.0   139.0   144.0   
2023-02-03   18             0.0  257.0  283.0    11.0   113.0   152.0   
2023-02-04  154             0.0   18.0  281.0    87.0   301.0    58.0   

            rolling_mean_7  rolling_mean_30  diff_rolling_mean_7_30  \
date                                                                  
2023-01-31      173.714286       145.066667               28.647619   
2023-02-01      187.428571       144.966667               42.461905   
2023-02-02      185.714286       148.733333               36.980952   
2023-02-03      147.857143       144.266667                3.590476   
2023-02-04      129.714286       147.466667              -17.7